In [1]:
# Import the RL algorithm (Trainer) we would like to use.



from custom_trainer import DeepingDQNTrainer

# Configure the algorithm.
config = {
    # Environment (RLlib understands openAI gym registered strings).
    "env": "CartPole-v0",
    # Use 2 environment workers (aka "rollout workers") that parallelly
    # collect samples from their own environment clone(s).
    "num_workers": 2,
    "num_gpus":1,
    # Change this to "framework: torch", if you are using PyTorch.
    # Also, use "framework: tf2" for tf2.x eager execution.
    "framework": "torch",
    "train_batch_size":32,
    # Tweak the default model provided automatically by RLlib,
    # given the environment's observation- and action spaces.
    "model": {
        "fcnet_hiddens": [64, 64],
        "fcnet_activation": "relu",
    },
    # Set up a separate evaluation worker set for the
    # `trainer.evaluate()` call after training (see below).
    "evaluation_num_workers": 1,
    # Only for evaluation runs, render the env.
    "exploration_config": {
        # The Exploration class to use.
        "type": "EpsilonGreedy",
        # Config for the Exploration class' constructor:
        "initial_epsilon": 1.0,
        "final_epsilon": 0.02,
        "epsilon_timesteps": 10000, 
    },
    ""
    "log_level": "DEBUG"
}

RAY_USE_MULTIPROCESSING_CPU_COUNT=1
def train(config, reporter):
    # Create our RLlib Trainer.
    trainer = DeepingDQNTrainer(config=config)
    while True:
        result = trainer.train()
        reporter(**result)
        if result['training_iteration'] > 5:
            break 
        print(result)
        print("hello")


# Run it for n training iterations. A training iteration includes
# parallel sample collection by the environment workers as well as
# loss calculation on the collected batch and a model update.
import ray
ray.tune.run(train, config=config, stop={"training_iteration": 5})

# Evaluate the trained Trainer (and render each timestep to the shell's
# output).
# trainer.evaluate()


2022-02-08 11:47:21,317	WARNING utils.py:534 -- Detecting docker specified CPUs. In previous versions of Ray, CPU detection in containers was incorrect. Please ensure that Ray has enough CPUs allocated. As a temporary workaround to revert to the prior behavior, set `RAY_USE_MULTIPROCESSING_CPU_COUNT=1` as an env var before starting Ray. Set the env var: `RAY_DISABLE_DOCKER_CPU_WARNING=1` to mute this warning.
2022-02-08 11:47:23,782	WARNING tune.py:570 -- Tune detects GPUs, but no trials are using GPUs. To enable trials to use GPUs, set tune.run(resources_per_trial={'gpu': 1}...) which allows Tune to expose 1 GPU to each trial. You can also override `Trainable.default_resource_request` if using the Trainable API.


Trial name,status,loc
train_CartPole-v0_e164c_00000,PENDING,


2022-02-08 11:47:23,933	ERROR syncer.py:111 -- Log sync requires rsync to be installed.
(ImplicitFunc pid=28887) 2022-02-08 11:47:26,591	WARNING trainer.py:1947 -- You have specified 1 evaluation workers, but your `evaluation_interval` is None! Therefore, evaluation will not occur automatically with each call to `Trainer.train()`. Instead, you will have to call `Trainer.evaluate()` manually in order to trigger an evaluation run.


Trial name,status,loc
train_CartPole-v0_e164c_00000,RUNNING,10.244.20.227:28887


In [ ]:
analysis = ray.tune.run(
    ppo.PPOTrainer,
    config=config,
    local_dir=log_dir,
    stop=stop_criteria,
    checkpoint_at_end=True)